In [10]:
# !pip install langchain==0.1.13 langchainhub==0.1.15 python-dotenv==1.0.1 huggingface_hub==0.22.2
# !pip install sentence-transformers
# !pip install groq
# !pip install langchain_groq
# !pip install google-search-results
# !pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.2 MB/s eta 0:00:00


In [2]:
from langchain.agents import load_tools
from groq import Groq
import string
import os

client = Groq(
    api_key = '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'
)

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent
llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")



In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


chain = prompt | llm


chain.invoke("how cna langsmith help with tetsing")

AIMessage(content="Langsmith, a cutting-edge AI-powered tool, can significantly enhance testing efforts by providing various benefits. Here are some ways Langsmith can help with testing:\n\n1. **Automated Test Data Generation**: Langsmith can generate high-quality, realistic test data, reducing the time and effort required to create test cases. This feature is particularly useful for testing complex systems that require large amounts of data.\n\n2. **Intelligent Test Scripting**: Langsmith's AI capabilities can assist in generating test scripts, reducing the manual effort required to write and maintain test cases. This enables testers to focus on higher-level tasks, such as test strategy and test data management.\n\n3. **Natural Language Processing (NLP) for Test Case Analysis**: Langsmith's NLP capabilities can analyze test cases, identifying patterns, and anomalies, and providing insights to improve test coverage and efficiency.\n\n4. **Automated Test Case Prioritization**: Langsmith

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input":"how can langsmith help with testing"})

"As a technical documentation writer, I'm excited to share with you how Langsmith can assist with testing!\n\nLangsmith is an innovative platform that leverages AI-powered language models to generate high-quality, human-like text. In the context of testing, Langsmith can be a game-changer in several ways:\n\n1. **Test Data Generation**: Langsmith can generate a vast amount of diverse, realistic, and context-specific test data, including input values, expected outputs, and edge cases. This can help you create more comprehensive and robust tests, reducing the likelihood of errors and increasing test coverage.\n2. **Automated Test Scripting**: Langsmith can be used to generate test scripts in various programming languages, such as Python, Java, or C++. By leveraging Langsmith's language generation capabilities, you can automate the creation of test scripts, saving time and effort while ensuring consistency and accuracy.\n3. **Test Scenario Creation**: Langsmith can assist in generating te

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
vector = FAISS.from_documents(documents,embeddings)


from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

{context}


Question: {input}""")

document_chain = create_stuff_documents_chain(llm,prompt)


from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)


response = retrieval_chain.invoke({"input":"how can langsmith help with testing"})
print(response["answer"])

Based on the provided context, LangSmith can help with testing in the following way:

1. **Evaluation**: LangSmith allows you to define a system to test, data to serve as test cases, and optionally evaluators to grade the results. This enables you to evaluate the performance of your AI system.
2. **Tracing**: LangSmith provides a way to log traces, which can help in debugging and testing your AI application. You can use the `traceable` function to auto-trace LLM calls in-context.
3. **Automated testing**: LangSmith's evaluation feature can be used to automate testing of your AI system. You can define evaluators to score the results of your system's outputs, and LangSmith will help you run the evaluation and provide results.

Overall, LangSmith provides a framework for testing and evaluating AI systems, making it easier to ensure their quality and performance.


In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation, generate a search query to look up in order to get information")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

[Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace‚ÄãWe provide multiple ways to log traces to LangSmith. Below, we'll highlight", metadata={'source': 'https://docs.smith.lang

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the user's questions based on the below context\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),

])
document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retriever_chain,document_chain)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

{'chat_history': [HumanMessage(content='Can Langsmith help test my LLm applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-a

In [14]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for info about LangSmith. For any questions about LangSmith, use this tool!"
)


from langchain_community.tools.tavily_search import TavilySearchResults


os.environ['TAVILY_API_KEY'] = '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'


search = TavilySearchResults(tavily_api_key = os.getenv('TAVILY_API_KEY'))


tools = [search]

In [16]:
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_structured_chat_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate
import typing
import langchain_core
#prompt = hub.pull("hwchase17/structured-chat-agent")
prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]
        },
    metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'structured-chat-agent', 'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'},
    messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'],
    template='Respond to the human as helpfully and accurately as possible. BE AS SPECIFIC AND DETAILED AS POSSIBLE. Your answer must be long and you must perform a giant web search, that is, not superficial. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per \
JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Your answer must be long and you need to perform enough searches with the searching tool to get enough information. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation. Please put all your observations into the final answer IT MUST BE DETAILED FINAL ANSWER')),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'],
    template='{input}\n\n{agent_scratchpad}\n (reminder to respond in a JSON blob no matter what)'))]
)
agent = create_structured_chat_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
system_msg = """
                You are an expert at questioning the client about their renovation or contstruction project quote.
                Please keep in mind all of these things when asking questions: ask the user specific info needed for the quote, such as:

                    1.Property Details:
                    Address and location of the property.
                    Type of property (e.g., residential, commercial).
                    Age and current condition of the property.
                    Square footage of the area to be renovated.
                    2.Scope of the Project:
                    Detailed description of the renovation work required.
                    Specific areas or rooms that need renovation.
                    Desired changes and additions (e.g., new rooms, extensions, wall removal).
                    Level of finishes desired (standard, premium, luxury).
                    3.Design Preferences:
                    Style or theme of the renovation (modern, traditional, industrial, etc.).
                    Color schemes and material preferences.
                    Any specific fixtures or features desired (e.g., type of flooring, lighting fixtures).
                    4.Budget:
                    Your budget range for the project.
                    Any flexibility within the budget.
                    5.Timeline:
                    Desired start date and completion date.
                    Any constraints or deadlines (e.g., events planned at the property).
                    Legal and Compliance Information:
                    Information on any permits or approvals already obtained.
                    Any restrictions or caveats affecting the property.
                    6.Utility Modifications:
                    Changes or improvements to electrical, plumbing, or HVAC systems.
                    Details of any provisions for appliances or systems (e.g., energy-efficient or smart home features).
                    7.Access & Logistics:
                    Access requirements for renovation or construction work (e.g., elevator access, restricted hours).
                    Logistic considerations necessary for the project (e.g., space for storing materials, parking for construction vehicles).
                    8.Special Requirements:
                    Access provisions for renovations or construction work done on the property.
                    Special considerations or instructions (e.g., conserving existing architectural features).
                    What specific materials they are looking at.
                    Information about other construction or renovation projects completed on the property.
                    IMPORTANT!: Make sure you get clear answers from the user that can actually be used for making the quote, and if you are not provided a clear answer ask to clarify, and restate the question and explain what it means.
                    IMPORTANT!: Ask each question that you have ONE BY ONE

                    IMPORTANT!: WHEN YOU HAVE ALL THE INFORMATION, JUST SAY 'hehehe' at the end.

                """


chat_history = []
while True:


    client_req = str(input())
    quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": client_req + "Chat History for your own context and info - DONT ASK A QUESTION IF THE ANSWER IS ALREADY IN CHAT HISTORY: " + str(chat_history)
            }
        ],

        model="llama3-70b-8192",
    )
    output = (quotebot.choices[0].message.content)
    print(output)
    chat_history.append(output)
    if('hehehe' in output.lower()): break

hello
Let's get started! I'll ask questions to gather information for a renovation or construction project quote.

First, can you please provide the address and location of the property?
3210 greenstone way Herndon VA 20171
You've already provided the address: 3210 Greenstone Way, Herndon, VA 20171. That's wonderful! 

Next question: What type of property is this? Is it a residential or commercial property?
it is literally a single family home
Since we've already established the address, let's move on to the next question. 

You've already mentioned it's a single-family home, so I'll ask the next question. What is the age and current condition of the property? Is it a relatively new build or an older home that may require more renovation work?
1980 it was built.
I see that we've already established some details about the property. Let's move on to the next question.

You mentioned the property was built in 1980. Can you please tell me approximately what is the square footage of the are

In [23]:
consultors_list = []
consultationbot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": "You are a proffesional renovation consultor. Based on the chat history, create a streamlined material plan for the bathroom, choosing what materials to use, and how much would be used and where. MAKE SURE TO BE VERY SPECIFIC in what materials you will use and how much. ALSO MAKE SURE TO PROVIDE A BLURB AT THE START OF THE RESPOnse And THEN A MATERIAL LIST AND HOW MUCH YOU WILL NEEED." + "Here is the chat history:" + str(chat_history)

            }
        ],
        model="llama3-70b-8192",
    )
consoltation_output = (consultationbot.choices[0].message.content)
consultors_list.append(consoltation_output)



quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": "Based on the chat history as well as the consultors list of materials, you must put all the required information for the quote into a streamlined format so that a web search query can be formed for it." + "Chat History for your own context and info: " + str(chat_history) + "Consultors List: " + str(consultors_list)
            }
        ],
        model="llama3-70b-8192",
    )
streamlined_output = (quotebot.choices[0].message.content)






In [24]:
streamlined_output

'Here is the streamlined format for the quote:\n\n**Property Details:**\nAddress: 3210 Greenstone Way, Herndon, VA 20171\nType of Property: Single-family home\nAge and Current Condition: Built in 1980\nSquare Footage of Area to be Renovated: TBD\n\n**Scope of the Project:**\nRenovation work required: Bathroom renovation, including updates to bathtub, HVAC system, and other areas\nSpecific areas or rooms that need renovation: Bathroom upstairs\nDesired changes and additions: Update fixtures, re-tile, and reconfigure layout\nLevel of finishes desired: Premium\n\n**Design Preferences:**\nStyle or theme: Modern\nColor scheme: TBD\nMaterial preferences: High-quality materials and features\nSpecific fixtures or features: Porcelanosa ceramic tile, Caesarstone quartz countertops, Kraus stainless steel undermount sink, Moen Eva faucet, Kohler Sterling shower enclosure, American Standard Americast bathtub, Hinkley Lighting chrome bathroom light fixtures, Halo recessed lighting kit, Toto Ultramax

In [26]:
consoltation_output

"**Material Plan for Bathroom Renovation**\n\nBased on our conversation, I've created a streamlined material plan for the bathroom renovation. Please find the details below:\n\n**Flooring:**\n\n* 100 sq. ft. of Porcelanosa ceramic tile flooring in a matte finish (Color: Grey Mosaic)\n* 1 box of Laticrete SpectraLOCK Pro Grout (Color: Bright White)\n\n**Walls:**\n\n* 400 sq. ft. of Porcelanosa ceramic wall tile in a matte finish (Color: White Matte)\n* 2 boxes of Laticrete SpectraLOCK Pro Grout (Color: Bright White)\n\n**Countertops:**\n\n* 60 sq. ft. of Caesarstone quartz countertops (Color: London Grey)\n* 1 box of Silestone quartz countertop sealant\n\n**Sink and Faucet:**\n\n* 1 x Kraus stainless steel undermount sink (Model: KBU14)\n* 1 x Moen Eva faucet (Model: EVS101)\n\n**Shower and Bathtub:**\n\n* 1 x Kohler Sterling shower enclosure (Model: K-7238)\n* 1 x American Standard Americast bathtub (Model: 2422)\n\n**Lighting:**\n\n* 2 x Hinkley Lighting chrome bathroom light fixtures

In [27]:
output = agent_executor.invoke({"input":"Given the chat history --> "+streamlined_output+"<-- AS WELL AS THE CONSULTANT'S INFORMATION -->" + consoltation_output + " --> look for labor and material costs for whatever the user asked for in the location of the user's property. maybe also use the costs of houses or properties very near to that location to decide on the cost. BE VERY SPECIFIC. LOTS OF NUMBERS"})



> Entering new AgentExecutor chain...
Action:```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "bathroom renovation costs in Herndon, VA"
  }
}
```[{'url': 'https://www.thumbtack.com/va/herndon/bathroom-remodeling', 'content': "What's happening behind your walls and under your floor will also affect bathroom remodeling cost. Rot, damage or mold can lead to additional work and charges. Here are some examples of average bathroom remodeling costs: 6-foot by 6-foot bathroom remodel: $3,500, including labor and materials. Standard renovation: $6,500-$10,000. Price ..."}, {'url': 'https://www.oakhillbuilding.com/bathrooms/Herndon/', 'content': 'Award winning remodeling contractor in Herndon specializing in bathroom remodeling. Call Oak Hill Building today for a free quote (703) 591-1200. ... Whether you are renovating a powder room or constructing a master suite bathroom in Herndon, VA, Oak Hill Building & Remodeling will guide you through every step of the d

In [29]:
output

{'input': "Given the chat history --> Here is the streamlined format for the quote:\n\n**Property Details:**\nAddress: 3210 Greenstone Way, Herndon, VA 20171\nType of Property: Single-family home\nAge and Current Condition: Built in 1980\nSquare Footage of Area to be Renovated: TBD\n\n**Scope of the Project:**\nRenovation work required: Bathroom renovation, including updates to bathtub, HVAC system, and other areas\nSpecific areas or rooms that need renovation: Bathroom upstairs\nDesired changes and additions: Update fixtures, re-tile, and reconfigure layout\nLevel of finishes desired: Premium\n\n**Design Preferences:**\nStyle or theme: Modern\nColor scheme: TBD\nMaterial preferences: High-quality materials and features\nSpecific fixtures or features: Porcelanosa ceramic tile, Caesarstone quartz countertops, Kraus stainless steel undermount sink, Moen Eva faucet, Kohler Sterling shower enclosure, American Standard Americast bathtub, Hinkley Lighting chrome bathroom light fixtures, Halo

In [32]:
output["output"]

{'response': "Based on the search results, here is a detailed breakdown of the estimated costs for the bathroom renovation project in Herndon, VA:\n\n**Flooring:**\n* 100 sq. ft. of Porcelanosa ceramic tile flooring in a matte finish (Color: Grey Mosaic): $3,500 - $5,000 (labor and materials)\n* 1 box of Laticrete SpectraLOCK Pro Grout (Color: Bright White): $100 - $200\n\n**Walls:**\n* 400 sq. ft. of Porcelanosa ceramic wall tile in a matte finish (Color: White Matte): $6,000 - $8,000 (labor and materials)\n* 2 boxes of Laticrete SpectraLOCK Pro Grout (Color: Bright White): $200 - $400\n\n**Countertops:**\n* 60 sq. ft. of Caesarstone quartz countertops (Color: London Grey): $3,000 - $4,000 (labor and materials)\n* 1 box of Silestone quartz countertop sealant: $100 - $200\n\n**Sink and Faucet:**\n* 1 x Kraus stainless steel undermount sink (Model: KBU14): $500 - $800\n* 1 x Moen Eva faucet (Model: EVS101): $200 - $400\n\n**Shower and Bathtub:**\n* 1 x Kohler Sterling shower enclosure (

In [33]:
newstr = ""
ctr = 0
for word in output["output"]["response"].split(" "):
    if(ctr % 6 == 0 and ctr != 0):
        newstr += "\n"
    newstr += word + " "
    ctr += 1

print(newstr)

Based on the search results, here 
is a detailed breakdown of the 
estimated costs for the bathroom renovation 
project in Herndon, VA:

**Flooring:**
* 100 sq. 
ft. of Porcelanosa ceramic tile flooring 
in a matte finish (Color: Grey 
Mosaic): $3,500 - $5,000 (labor and 
materials)
* 1 box of Laticrete SpectraLOCK 
Pro Grout (Color: Bright White): $100 
- $200

**Walls:**
* 400 sq. ft. of 
Porcelanosa ceramic wall tile in a 
matte finish (Color: White Matte): $6,000 
- $8,000 (labor and materials)
* 2 
boxes of Laticrete SpectraLOCK Pro Grout 
(Color: Bright White): $200 - $400

**Countertops:**
* 
60 sq. ft. of Caesarstone quartz 
countertops (Color: London Grey): $3,000 - 
$4,000 (labor and materials)
* 1 box 
of Silestone quartz countertop sealant: $100 
- $200

**Sink and Faucet:**
* 1 x 
Kraus stainless steel undermount sink (Model: 
KBU14): $500 - $800
* 1 x 
Moen Eva faucet (Model: EVS101): $200 
- $400

**Shower and Bathtub:**
* 1 x 
Kohler Sterling shower enclosure (Model: K-

In [34]:
quotebot2 = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": "You are an expert in PREPARING A REAL ESTATE QUOTE IN PROPER FORMAT from ONLY what is in the user's request, given a web search synthesis as one input and user's request as another input"

            },

            {
                "role": "user",
                "content": "Based on the input, which is <<<"+newstr+">>> you must parse the input for the renovation quote so that your response talks about ONLY the stuff relevant to the user's quote request. IMPORTANT: GIVE AN EXACT AMOUNT FOR MATERIAL VALUE & LABOR COST " + "USer's request for your own context and info: " + str(output)
            }
        ],
        model="llama3-70b-8192",
    )
output2 = (quotebot2.choices[0].message.content)
print(output2)

Here is the prepared real estate quote in proper format:

**Property Details:**
Address: 3210 Greenstone Way, Herndon, VA 20171
Type of Property: Single-family home
Age and Current Condition: Built in 1980
Square Footage of Area to be Renovated: TBD

**Scope of the Project:**
Renovation work required: Bathroom renovation, including updates to bathtub, HVAC system, and other areas
Specific areas or rooms that need renovation: Bathroom upstairs
Desired changes and additions: Update fixtures, re-tile, and reconfigure layout
Level of finishes desired: Premium

**Design Preferences:**
Style or theme: Modern
Color scheme: TBD
Material preferences: High-quality materials and features
Specific fixtures or features: Porcelanosa ceramic tile, Caesarstone quartz countertops, Kraus stainless steel undermount sink, Moen Eva faucet, Kohler Sterling shower enclosure, American Standard Americast bathtub, Hinkley Lighting chrome bathroom light fixtures, Halo recessed lighting kit, Toto Ultramax II toil